***
# LITERATURE
***

***
# Deep reinforcement learning for turbulent drag reduction in channel flows
***
> Guastoni, L., Rabault, J., Schlatter, P., Azizpour, H., & Vinuesa, R. (2023). Deep reinforcement learning for turbulent drag reduction in channel flows. The European Physical Journal. E, Soft Matter, 46.
> 
> Latex :
> > @article{Guastoni2023DeepRL,
  title={Deep reinforcement learning for turbulent drag reduction in channel flows},
  author={Luca Guastoni and Jean Rabault and Philipp Schlatter and Hossein Azizpour and Ricardo Vinuesa},
  journal={The European Physical Journal. E, Soft Matter},
  year={2023},
  volume={46},
  url={https://api.semanticscholar.org/CorpusID:256194495}
}

## Goal 
Enhance drag reduction with flow control using RL.

## How 
$$\text{DRL with MARL}$$

![Figure 1 du papier](Images/Literature/MARL_DRL_Lucas.png)

- **Control Policy and Evaluation:**
  - Each agent computes wall-normal velocity intensity to be applied.
  - Control policy evaluated over fixed time interval.
  - Actual control varies linearly from old value to new one to avoid numerical instabilities.
  - Time units scaled with $t^* = \frac{\nu}{u_{\tau}^2}$, where $\Delta t^+ = 0.6$ in numerical experiments.
  - Actuation value limited to range $[-u_{\tau}, u_{\tau}]$.
 
- **Observation for Agent:**
  - Agent observes portion of one or more wall-parallel planes of sampled flow quantities.
  - Observable portion of flow field above actuation.
  - Sampling heights referred using inner-scaled wall-normal coordinates: $y^+ = \frac{y u_{\tau}}{\nu} = \frac{y}{l^*} \in [0, Re_{\tau}]$, where $l^*$ is viscous length.
 
- **Deep Reinforcement Learning Algorithm (DDPG):**
  - Optimizes deep neural network (actor) approximating relation between state and action (policy function).
  - Policy deterministic after learning.
  - Second neural network (critic) approximates action-value function $Q(s, a)$.
  - Simple fully connected networks used for both actor and critic.
    - Actor: Single layer of 8 nodes.
    - Critic: Three layers with 16, 64, and 64 nodes.
  - Gaussian noise with zero mean and variance $0.1u_{\tau}$ added to actions computed by actor network for exploration.
  - Policy updated every $\Delta t^+ = 180$, with 64 mini-batch gradient updates.
  - Mini-batches sampled from buffer replay including 5,000,000 (state, action, reward) tuples.
  - Default parameter values from Stable-baselines implementation used.
  - Configuration details available in associated code repository.

### Simualtion Setup 

- **Simulation Domains:** 
  - **Minimal Channel:** Ω = $2.67h \times h \times 0.8h$
  - **Larger Channel:** Ω = $2\pi h \times h \times \pi h$


- **Velocity Components:** 
  - $(x, y, z)$ represent streamwise, wall-normal, and spanwise directions respectively.
  - $(u, v, w)$ represent corresponding velocity components.


- **Friction Reynolds Number:** 
  - $Re_{\tau} = \frac{u_{\tau}h}{\nu}$
  - $u_{\tau} = \frac{\tau_{w}}{\rho}$
  - $\tau_{w}$ is the wall-shear stress, $\rho$ is fluid density, and $\nu$ is kinematic viscosity.


- **Reynolds Number:** 
  - $Re_{\tau} = 180$ in both domains.

- **Solver:** SIMSON (DNS coded in FORTRAN 77/90)
  - Pseudo-spectral code using Chebyshev polynomials in wall-normal direction.


- **Resolution:**
  - **Minimal Channel:** $16 \times 65 \times 16$
  - **Larger Domain:** $64 \times 65 \times 64$


- **Time-Advancement Scheme:** 
  - Second-order Crank-Nicholson for linear terms.
  - Third-order Runge-Kutta for nonlinear terms.



How to deal with the solver-DRL Interface

- **Reinforcement Learning Framework and Environment:**
  - Framework: Stable-baselines 
  - Environment: Custom PettingZoo/Gym environment
  - Both written in Python.
  - Fluid solver coded in FORTRAN 77/90.

- **Interface Between Programming Languages:**
  - Interface based on message-passing interface (MPI) in this case.
  - Fluid solver spawned as child process of main learning process.
  - Intercom created between solver and Python main program.
  - Communication requests handled using five-character strings:
    - **STATE:** Request simulation to communicate state to agent.
    - **CNTRL:** Agent communicates new action to environment, updating controllable parameters in fluid solver.
    - **EVOLVE:** Compute several time iterations of solver after CNTRL update to observe effect of chosen action on environment. Instantaneous wall-shear stress passed to Python program for reward computation.
    - **TERMN:** Interrupt solver and close FORTRAN program. Used at end of each episode before restarting environment. Initial condition same for every episode unless stated otherwise.

- **Functions for Gym Environment API:**
  - Combination of messages used to define required functions.


## Validation and Baseline

- **Opposition Control as Baseline:**
  - Established control benchmark for assessment of DRL-control laws.
  - Actuation at wall $v_{\text{wall}}$ by opposition control computed using expression:
    $v_{\text{wall}}(x,z,t) = -\alpha v(x,y_s,z,t) - \langle v(x,y_s,z,t) \rangle$
  - $\langle v(x,y_s,z,t) \rangle$: Spatial mean of wall-normal velocity field.
  - Second term enforces zero-net-mass-flux within domain.
  - $\alpha$: Positive scaling parameter fixed in spatial directions and time (typical value $\alpha = 1$).

- **Validation of Code Implementation:**
  - Reproduced opposition control strategy in solver.
  - Environment evolved with constant $\alpha$ until drag-reduction rate reached stationary value.
  - Previous studies reported highest drag reduction at $y^+ = 10$.
  - Computational domain size: $\Omega = L_x \times L_y \times L_z = 4\pi h \times h \times \frac{4}{3}\pi h$, resolution $N_x \times N_y \times N_z = 128 \times 129 \times 128$.
  - Friction Reynolds number $Re_{\tau} = 180$, Reynolds number based on centreline velocity $Re_{cl} = 3273$.
  - Drag reduction in literature: $\approx 14\%$ at $y^+ = 10$.
  - Their result in larger channel: $17.73\%$, acceptable agreement considering lower resolution and small difference in spanwise size of domain.

- **Validation with Different Sampling Planes:**
  - Ref. [50]: Sampling at $y^+ = 15$, $Re_{cl} = 3240$.
    - Drag reduction: $\approx 25.67\%$, very good agreement.
  - Ref. [52]: Sampling at $y^+ = 15$, $Re_{\tau} = 180$.
    - Drag reduction: $\approx 24\%$, good agreement with our result.
  
- **Overall Validation and Baseline:**
  - Results match those reported in literature, validating simulation setup and control implementation.
  - Drag reduction obtained with opposition control used as baseline for comparison with DRL results.


## Results

- **Traditional Control Strategy:**
  - Achieves approximately 20% drag reduction.

- **Current Approach:**
  - **Minimal Channel:** Achieves approximately 40% drag reduction.
  - **Larger Channel:** Achieves approximately 30% drag reduction.

### Considerations:
- The RL strategy incurs a slightly higher computational cost, although negligible in comparison to the achieved drag reduction.

- Investigation into the effects of varying Reynolds number (Re) on the obtained results is warranted.

## How to Utilize this Paper for Our Needs?

- **Baseline Comparison:**
  - Use the results presented in this paper as a baseline for evaluating our own codes.
  - Ensure our codes are tested with exactly the same parameters as described in this paper.
  - **Reference to outperfom**.

This approach allows for a direct comparison of our results with those reported in the paper, providing a valuable benchmark for assessing the performance and accuracy of our codes.

*** 
# Identifying regions of importance in wall-bounded turbulence through explainable deep learning
***


> Cremades, A., Hoyas, S., Deshpande, R., Quintero, P., Lellep, M., Lee, W.J., Monty, J.P., Hutchins, N., Linkmann, M., Marusic, I., & Vinuesa, R. (2023). Identifying regions of importance in wall-bounded turbulence through explainable deep learning.
> 
> Latex :
> > @inproceedings{Cremades2023IdentifyingRO,
  title={Identifying regions of importance in wall-bounded turbulence through explainable deep learning},
  author={Andres Cremades and Sergio Hoyas and Rahul Deshpande and Pedro Quintero and Martin Lellep and Will Junghoon Lee and Jason P. Monty and Nicholas Hutchins and Moritz Linkmann and Ivan Marusic and Ricardo Vinuesa},
  year={2023},
  url={https://api.semanticscholar.org/CorpusID:256504028}
}

## Importants theoretical notions in the article

### What is a coherrent structure ? 

A coherent structure in fluid dynamics refers to a specific pattern or organization of fluid motion that persists over time, despite the inherently chaotic nature of fluid flow. These structures are identifiable entities within the flow that exhibit some degree of organization or coherence.

**Definition:**
- A coherent structure is a specific pattern or organization of fluid motion that persists over time despite the inherent chaotic nature of fluid flow.

**Examples:**
- Vortices (tourbillons): Swirling regions of fluid motion characterized by a rotational flow pattern.
- Eddies (remous): Whirling motions of fluid, typically found downstream of obstacles or in turbulent flows.
- Jets (jets): High-velocity streams of fluid ejected into a surrounding medium.
- Wakes (sillages): Regions of disturbed flow downstream of solid objects or obstacles.
- Boundary layers (couche limites): Layers of fluid adjacent to a solid boundary exhibiting distinct flow characteristics.

**Literature Examples:**
- Streamwise streaks: Regions within a fluid flow where the velocity of fluid particles align predominantly in the direction of the flow, known as the streamwise direction. These streaks appear as elongated regions of higher or lower velocity compared to the surrounding fluid. [3]
- Reynolds-stress events: Fluctuations in the stress tensor components caused by turbulent fluctuations in velocity in a fluid flow. [22]
- Q-event (object of this paper): [This would need a specific definition or reference to explain further.]

Coherent structures play a crucial role in understanding and analyzing fluid flows, providing insights into the dynamics and behavior of complex flow phenomena.

#### Extra explanation from Marco thesis p29-30

> Atzori, M. (2021). Coherent structures and control in wall-bounded turbulent flows (Doctoral dissertation, KTH Royal Institute of Technology).
> > 
> Latex :
> > @phdthesis{atzori2021coherent,
  title={Coherent structures and control in wall-bounded turbulent flows},
  author={Atzori, Marco},
  year={2021},
  school={KTH Royal Institute of Technology}
}

- **Coherent Structures in Turbulent Flows:**
  - Incompressible turbulent flows share common governing equations and general properties.
  - Distinct phenomena arise due to geometrical constraints and symmetry properties in different flows.
  - Intense Reynolds-stress events in square/rectangular ducts and APG turbulent boundary layers were studied.
  - **Definition of Intense Reynolds-Stress Events:**
    - Connected regions with instantaneous velocity fluctuations higher than a threshold.
    - Based on root-mean-square of velocity components.
    - Extension of classic quadrant analysis by Lozano-Durán et al. (2012).
  - **Threshold Conditions:**
    - Criteria for off-diagonal components of Reynolds-stress tensor: 
      $ |uv| > H_{uv}u_{rms}v_{rms}, \, |uw| > H_{uw}u_{rms}w_{rms}, \, |vw| > H_{vw}v_{rms}w_{rms} $
    - Scalar thresholds $ H_{XX} $ ("hyperbolic hole") with normalization terms $ u_{rms}v_{rms} $, $ u_{rms}w_{rms} $, $ v_{rms}w_{rms} $.
    - Chosen for homogeneous detection of coherent structures.
  - **Percolation Analysis:**
    - Introduced by Moisy & Jiménez (2004) and used by Lozano-Durán et al. (2012).
    - Evaluates effectiveness of threshold with the ratio: 
      $ \frac{V_{max}}{V_{all}} $
    - Indicates transition between homogeneous and non-homogeneous flow.
  - **Percolation Crisis:**
    - Common in systems with high degrees of freedom and specific connectivity/statistical properties.
    - Used in fluid mechanics beyond structure identification, e.g., laminar-to-turbulent transition.
  - **Statistical Properties and Percolation Crisis:**
    - Paper 12 discusses the relationship between statistical properties of a scalar field and percolation crisis.


### What is Reynolds Stress ?

Reynolds stress ($\tau_{ij}$) is a fundamental concept in fluid mechanics, particularly in the study of turbulent flows. It represents the statistical average of the fluctuating component of the momentum transport caused by turbulent velocity fluctuations within a fluid flow. 

**Definition:**
Reynolds stress is the average stress caused by fluctuations in velocity in a turbulent flow. It arises due to the interaction between turbulent eddies of different sizes, leading to variations in momentum transfer within the flow.

**Reynolds Decomposition:**
The Reynolds decomposition separates the velocity field into its mean ($\overline{u_i}$) and fluctuating ($u_i'$) components:
$$
u_i = \overline{u_i} + u_i'
$$

**Hypothesis for Writing Reynolds Stress:**
- The Reynolds decomposition assumes that the flow variables can be decomposed into a mean part and a fluctuating part.
- The Reynolds stress tensor $\tau_{ij}$ is defined as the average of the product of the fluctuating velocities $u_i'$ and $u_j'$:
$$
\tau_{ij} = \overline{u_i' u_j'}
$$

**Components:**
Reynolds stress has both normal and shear stress components, representing momentum transfer in different directions within the flow. 
- The normal stress components represent momentum transfer perpendicular to the flow direction.
- The shear stress components represent momentum transfer parallel to the flow direction.

**Importance:**
Reynolds stress is crucial for understanding turbulence dynamics and predicting turbulent flow behavior in various engineering and scientific applications. It influences the overall momentum transfer, heat transfer, and mixing processes within turbulent flows.

**Measurement:**
Reynolds stress can be measured experimentally using various techniques such as hot-wire anemometry, particle image velocimetry (PIV), and laser Doppler velocimetry (LDV). These experimental methods allow researchers to quantify the magnitude and distribution of Reynolds stress in turbulent flows.

**Modeling:**
In computational fluid dynamics (CFD) simulations, Reynolds stress is often modeled using turbulence models such as the Reynolds-averaged Navier-Stokes (RANS) equations or large eddy simulation (LES) models. These models aim to predict the behavior of turbulent flows by simulating the statistical characteristics of Reynolds stress.


### What are Q-Events ?

Based on reference [22], it serves as the primary energy source for all structures within turbulent flow. Hence, its study holds significant intrigue and importance.

**Definition**: Q-events denote distinct flow regions intricately linked with momentum transfer and the generation of turbulent kinetic energy (TKE), invariably contributing to substantial drag within the system.

**Important properties**: 
- These are delineated as coherent zones exhibiting instantaneous high Reynolds stress.
- Their identification hinges on the condition:
$$u(x, y, z, t)v(x, y, z, t) > Hu'(y)v'(y),$$
where $H$ represents a hyperbolic function. (This criterion serves as a crucial tool for detection.)
- Q-events effectively highlight regions characterized by statistically significant magnitudes of Reynolds stress.
- Quadrant analysis categorizes them into four distinct types:
  1. **Outward Interactions:** Where $u' > 0$ and $v' > 0,$
  2. **Ejections:** Featuring $u' < 0$ and $v' > 0,$
  3. **Inward Interactions:** With $u' < 0$ and $v' < 0$,
  4. **Sweeps:** Characterized by $u' > 0$ and $v' < 0$.

Understanding these coherent structures not only provides insights into the dynamics of turbulent flows but also facilitates the development of strategies for managing associated drag and optimizing various processes.

#### Extra explanation from Luca thesis p32-33

> Guastoni, L. (2023). Time, space and control: deep-learning applications to turbulent flows (Doctoral dissertation, KTH Royal Institute of Technology).
> > 
> Latex :
> > @phdthesis{guastoni2023time,
  title={Time, space and control: deep-learning applications to turbulent flows},
  author={Guastoni, Luca},
  year={2023},
  school={KTH Royal Institute of Technology}
}

<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/quadrant-analysis.png" alt="Figure 1 du papier" width="600">
</div>

The $(v',u')$ plot is called the quadrant analysis.

- Quadrant analysis is a tool used to analyze controlled flow in various strategies.
- Introduced by Wallace et al. (1972) to categorize contributions to Reynolds shear stresses.
- Divides streamwise and wall-normal fluctuations into four categories: Q1, Q2, Q3, Q4.
- Q2 and Q4 events correspond to ejections and sweeps in turbulent flows.
- Predominance of Q4 near the wall and Q2 farther from the wall in wall-bounded turbulence.
- Q1 and Q3 interactions are outward and inward, respectively, and less frequent.
- Q2 and Q4 motions are gradient-type interactions related to vertical momentum fluxes.
- Q1 and Q3 are counter-gradient-type interactions.
- Effective in revealing the connection between fluctuation distribution and turbulent features.
- Used to identify how different control strategies affect flow physics.



### About the Paper:

<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/Andres_Workflow.png" alt="Figure 1 du papier" width="400">
</div>



**Objective:** The aim is to utilize U-nets for predicting the evolution of turbulent channel flow.

**Data:** The study is based on 6,000 instantaneous realizations obtained from simulations of turbulent channel flow.

**Workflow:**
- **Domain Segmentation:** The domain is segmented into Q events.
- **Feature Extraction:** Each structure is considered as an input feature for the SHAP algorithm.
- **SHAP Algorithm:** Game-theoric method that calculates the importance of each feature on the U-net prediciton.

**Process:**
- **Flow Prediction:** Utilization of a U-net for flow prediction.
- **Structure Evolution:** Advancing a time step in the simulation to determine structure evolution.
- **SHAP Values:** Quantification of the importance of each coherent turbulent structure using SHAP values.
- **Comparison:** Comparison of the predicted solution with the simulated flow field in the next time step.

**Analysis:**
- Identification of coherent structures representing ejections and sweeps.
- Utilization of Explainable AI (XAI) to define an objective metric, confirming and extending previous results.

**Previous Approaches:**
- **Encinar and Jiménez [33]:** Studied homogeneous isotropic turbulence by introducing perturbations and numerically integrating governing equations.
- **Lozano-Durán and collaborators [34, 35]:** Employed information theory to identify regions of flow with the highest impact on future flow states.

**Advantages of Present Approach:**
- Reliance on deep learning provides a good surrogate for governing equations, even in environments where full access to DNS data is not possible, such as experiments.
- Allows assessment of important flow regions at higher Reynolds numbers and with less available data.

**Importance:** The significance of this approach is demonstrated in its applications in flow prediction and flow control.

**Simulation parameters:**
- $Re_{\tau} = 125$,
- Poiseuille flow : **Channel** dimensions Lx = 2πh, Lz = πh, with streamwise periodicity. (h ? ,y ?)


***
# More about coherent structures from Marco paper 
***

# Geometry and clustering of intense structures in isotropic turbulence

"(Moisy & Jiménez 2004) introduced the percolation analysis in their study of vortical and dissipative coherent structures in homogeneous isotropic turbulence", Marco.

> MOISY, F., & JIMÉNEZ, J. (2004). Geometry and clustering of intense structures in isotropic turbulence. Journal of Fluid Mechanics, 513, 111–133. doi:10.1017/S0022112004009802
> 
> Latex :
> > @article{MOISY_JIMÉNEZ_2004, title={Geometry and clustering of intense structures in isotropic turbulence}, volume={513}, DOI={10.1017/S0022112004009802}, journal={Journal of Fluid Mechanics}, author={MOISY, F. and JIMÉNEZ, J.}, year={2004}, pages={111–133}} <div></div>


#### What I want to learn from this paper 
What is a percolation analysis ?

## Quick reading 

*From 1.Introduction*
### Main Points:
- The paper addresses the interplay between geometric aspects and statistical properties of intense vortical structures in turbulent flows.
- It discusses the difficulty in characterizing intermittency in turbulent flows, either from the geometry of structures or from statistical fluctuations at different scales.
- The paper highlights the challenges in connecting statistical quantities with the geometry of individual structures and their spatial distribution.
- Various criteria and methods for identifying intense vortical structures are discussed, including pressure criteria, eigenvalues of the velocity gradient tensor, and fractal analysis.
- Experimental and numerical evidence suggests non-random spatial distributions and clustering of intense vortical structures.
- The paper aims to investigate the links between fractal and structural descriptions of turbulence.

### Variables and Definitions:
1. **Vorticity Magnitude ($\omega$)**: Measure of the local rotation of fluid particles in a flow.
2. **Rate-of-Strain Tensor ($S_{ij}$)**: Describes the stretching and deformation of fluid elements in a flow.
3. **Pressure ($p$)**: Represents the scalar field associated with the force exerted by a fluid.
4. **Eigenvalues of the Velocity Gradient Tensor ($\lambda_1, \lambda_2, \lambda_3$)**: Represent the magnitudes of the stretching and rotation of fluid elements in different directions.
5. **Fractal Dimension ($D$)**: Measure of the complexity of a geometric structure in terms of its scaling properties.
6. **Multifractal Exponents**: Describe the scaling behavior of different parts of a multifractal structure.

### Formulas:
1. **Pressure Poisson Equation**: $\nabla^2 p / \rho = Q = 1/2 \omega^2 - S_{ij}S_{ij}$
   - Explanation: Represents the balance between vorticity and strain rate in determining pressure distributions.
2. **Fractal Dimension Formula**: $D \approx 2.7$
   - Explanation: Provides a measure of the complexity of the spatial distribution of intense vortical structures.


*From 2.Data Generation*

### Main Points:
- The paper analyzes numerical simulations conducted by Jiménez et al. (1993) to study turbulent flows.
- Data consists of two instantaneous velocity fields of three-dimensional forced turbulence with 512^3 collocation points and periodic boundary conditions.
- Resolution in physical space is achieved with a scale separation of approximately 200 (L0/η).
- Quantities of interest include vorticity magnitude and strain-rate magnitude.
- Statistical and geometrical properties of sets defined by thresholding vorticity and strain rate fields are studied.

### Variables and Definitions:
1. **Vorticity Magnitude ($|\omega|$)**: Measure of the local rotation of fluid particles in a flow.
2. **Strain-Rate Magnitude ($|s|$)**: Describes the stretching and deformation of fluid elements in a flow.
3. **Energy Dissipation Rate ($\varepsilon(x)$)**: Rate at which energy is dissipated in the flow, calculated from the strain-rate magnitude.
4. **Kinematic Viscosity ($\nu$)**: Measure of fluid's resistance to deformation.
5. **Root-Mean-Square Fluctuations ($\omega'$ and $s'$)**: Represent the fluctuations of vorticity and strain rate, respectively.
6. **Mean Dissipation Rate ($\langle \varepsilon \rangle$)**: Average dissipation rate of energy in the flow.

### Formulas:
1. **Local Energy Dissipation Rate**: $\varepsilon(x) = 2\nu|s(x)|^2$
   - Explanation: Computes the rate of energy dissipation at a specific point in the flow based on the strain-rate magnitude.
2. **Vorticity and Strain-Rate Balance**: $\omega'^2 = 2s'^2 = \langle \varepsilon \rangle / \nu$
   - Explanation: Represents the balance between vorticity and strain rate on average in the flow.



*From 6.Concluding remarks*

**Main Points:**
- Box-counting methods applied to high vorticity and strain rate sets in isotropic turbulence at Reλ ≃ 168.
- No clear fractal behavior observed due to limited scale range.
- Minimum of logarithmic slope of box counts provides upper bound for possible fractal dimension.
- Local exponents provide estimates for fractal dimension of vorticity and dissipation field.
- Intense sets split into disconnected structures for high thresholds (< 4% volume fraction).
- Fractal properties influenced by structure shape, size distribution, and arrangement of baricenters.
- Strain-rate structures resemble flat sheets or ribbons, vorticity structures vary from ribbons to tubes.
- Intense structures not uniformly distributed, but clustered into inertial size clusters.
- Potential for scale-independent nesting of intense structures at higher Reynolds numbers.
- Decrease in global dimension with increased threshold affects one-point measurements.
- Dimensions for highest thresholds decrease to low values (~0.2), suggesting possible isolated objects.
- Different behavior observed for dynamical and passive fields.

**Variables and Definitions:**
- Reλ: Reynolds number.
- Dα∗: Minimum of logarithmic slope of box counts, upper bound for fractal dimension.
- D∗: Local exponents providing estimates for fractal dimension.
- |ω|, ω′, |s|, s′: Magnitude and fluctuations of vorticity and strain rate.
- Dω, Ds: Individual dimensions for vorticity and strain-rate structures.
- Aspect ratios: Ratio of structure dimensions.
- Baricenters: Centers of mass of structures.
- τ: Threshold for one-point measurements.
- δr: Length of free intervals in sets.

**Formulas:**
1. $D_{\alpha^*}$: Minimum of logarithmic slope of box counts.
2. $D^*$: Local exponents providing estimates for fractal dimension.
3. $D_{\omega}$, $D_{s}$: Individual dimensions for vorticity and strain-rate structures.
4. $\tau$: Threshold for one-point measurements.
5. $\delta r$: Length of free intervals in sets.

# The three-dimensional structure of momentum transfer in turbulent channels

"and Lozano-Dur ́an et al. (2012) employed it to qualitatively assess the effectiveness of the threshold.", p.39, Marco.

> Lozano-Durán, A., Flores, O., & Jiménez, J. (2012). The three-dimensional structure of momentum transfer in turbulent channels. Journal of Fluid Mechanics, 694, 100–130. doi:10.1017/jfm.2011.524
> 
> Latex :
> > @article{Lozano-Durán_Flores_Jiménez_2012, title={The three-dimensional structure of momentum transfer in turbulent channels}, volume={694}, DOI={10.1017/jfm.2011.524}, journal={Journal of Fluid Mechanics}, author={Lozano-Durán, Adrián and Flores, Oscar and Jiménez, Javier}, year={2012}, pages={100–130}} <div></div>


#### What I want to learn from this paper 
What is a percolation analysis ?



# Quick reading 

## History of the statistical properties of the Reynolds stresses

### Beginning p. 101
The study of the statistical properties of the Reynolds stresses began in the late 1960s, with the observation by Kline et al. (1967) that the near-wall low-velocity streaks undergo a process of lift-up, oscillation, break-up and ejection, which they called bursting. Kim, Kline & Reynolds (1971) showed that most of the turbulence production in the near-wall region occurs during those bursts, and several conditional- sampling techniques were developed to identify the structures involved in the process. Examples are the u-level detection of Lu & Willmarth (1973), the VITA (variable- interval time average) of Blackwelder & Kaplan (1976) and the VISA (variable- interval space average) of Kim (1985). Several of those techniques were surveyed by Bogard & Tiederman (1986), who concluded that the best compromise between detection probability and false positives was provided by the quadrant analysis of Wallace, Eckelman & Brodkey (1972), Willmarth & Lu (1972) and Lu & Willmarth (1973), in which points of the flow are classified in terms of the quadrant of the parameter plane of streamwise and wall-normal velocity fluctuations. Q1 events (outward interactions) have u > 0 and v > 0, Q2 events (ejections) have u < 0 and v > 0, Q3 events (inward interactions) have u < 0 and v < 0 and Q4 events (sweeps) have u > 0 and v < 0. Hereafter, Q events will be referred to simply as Qs, and we will occasionally group the ‘gradient’ Q2s and Q4s as Q−, and the ‘counter-gradient’ Q1s and Q3s as Q+.

### After p. 101

Although most of the early work focused on bursting in the buffer region, studies during the 1990s began to establish a relationship between near-wall bursting and outer structures. Antonia, Bisset & Browne (1990) applied a variant of the u-level method (WAG, window average gradient) to boundary layers with relatively high Reynolds numbers based on the momentum thickness, Reθ = U∞θ/ν = 1360–9630, corresponding to h+ = 530–3100. They reported that outer-scale discontinuities in the streamwise velocity, extending across the whole thickness of the boundary layer, tend to be located near the upstream end of the near-wall bursts. Wark & Nagib (1991) applied quadrant analysis to a boundary layer at Reθ = 4650 (h+ = 1500), and found large conditional events with sizes of the order of h. Using a related identification technique in the atmospheric boundary layer, Narasimha et al. (2007) described Reynolds-stress structures with lengths comparable to the distance to the wall that, because of the high Reynolds number involved, were both very long in wall units and very short compared with the boundary layer thickness. Hoyas & Jime ́nez (2006) showed that the spectral signatures of the very-large motions (Jime ́nez 1998; Kim&Adrian1999),orglobalmodes(delA ́lamoetal.2004),characteristicofthe outer region are present at the wall, as well as those of similar smaller structures in the logarithmic layer, and Hutchins & Marusic (2007b) showed that part of that influence is a modulation of the intensity of the small-scale buffer-layer fluctuations.

### Recntly p. 102


More recently, Flores & Jime ́nez (2010a) showed that the hierarchy of structures of increasing sizes postulated by Townsend (1961) for the logarithmic layer can be represented numerically in computational boxes of widths proportional to the height to be studied. Each box contains a single complex structure, including a segment of a streamwise velocity streak and a cluster of vortices, that bursts quasiperiodically to fill the box with essentially normal turbulence, with a strong Q2 and Q4 occurring simultaneously. The existence of a hierarchy of Q2s and Q4s was also conjectured by Wark & Nagib (1991), and is consistent with the similarity observed by Kailas & Narasimha (1994) between the conditionally sampled structures extracted from the near-wall region of laboratory boundary layers and those in the logarithmic region of the atmospheric boundary layer.
Q2s and Q4s play important roles in most of the structural models proposed to explain how turbulent kinetic energy and momentum are redistributed in wall-bounded turbulence. The majority of these models (see the review by Robinson 1991) are loosely based on the attached-eddy hypothesis of Townsend (1961) and involve wall- attached vortical loops growing from the wall into the outer region (Perry, Henbest & Chong 1986). More recently, in order to account for the very large-scale motions mentioned above, as well as for earlier experimental evidence on the internal structure of the bursts (Bogard & Tiederman 1986), a variant model has evolved to include vortex packets (Adrian, Meinhart & Tomkins 2000), in which trains of hairpin vortices grow from the wall by a process of merging (Tomkins & Adrian 2003) and self- regeneration (Zhou et al. 1999). The heads of the hairpins are arranged into inclined shear layers generated by the combined induction of the hairpins of the packet, with a Q4 upstream and above the packet and a Q2 beneath it. Ganapathisubramani, Longmire & Marusic (2003) used PIV data on wall-parallel planes in a boundary layer with Reθ = 2500 (h+ = 1060) to quantify the contribution to the Reynolds stresses by packets of hairpins. They report that, at y+ = 100, packets covering 4 % of the area produce 28 % of the total Reynolds stresses.
The hairpin-packet model is usually understood to involve relatively smooth vortex loops as the structures associated with Townsend’s (1976) attached eddies, although recent discussions tend to include ‘canes, heads, legs and three-quarter hairpin shapes, generally asymmetric and distorted’ into the class of ‘hairpin-like’ vortices (Dennis & Nickels 2011a), making them hard to distinguish from generic vortices. An alternative,lessorganized,structurewasproposedbydelA ́lamoetal.(2006),who analysed the statistical properties of clusters of vortices (clusters hereafter) in direct numerical simulations (DNSs) of turbulent channels with h+ = 550–1880. The clusters segregate into wall-attached and wall-detached families. The wall-detached clusters are dissipative objects, with sizes that scale with the local Kolmogorov scale. The attached ones form a self-similar family of objects that mark strong Q2s. The averaged flow conditioned to an attached cluster consist on an elongated low-velocity streak extending downstream of the cluster, flanked by a pair of shorter high-velocity streaks. While this averaged structure is consistent with the hairpin packet paradigm, del A ́lamoetal.(2006)pointedoutthattheinstantaneousclustersaremorecomplex,in agreement with earlier visualizations by Tanahashi et al. (2004).

### About $H$ and percolation diagram p. 105

<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/percolation_diagram.png" alt="Figure 1.a; du papier" width="700">
</div>

Note that the threshold in (2.1) depends on the wall distance. As noted by Nagasoa & Handler (2003), using an identification method with a constant threshold, such as |τ| > Hu′(y0)v′(y0), with some reference y0, is problematic in inhomogeneous flows. In our case, very few objects are recovered in the outer region if y0 is chosen in the buffer layer, while choosing a high y0 results in a confusingly cluttered buffer layer. This behaviour agrees with the observations of Blackburn, Mansour & Cantwell (1996) and worsens as Re increases. Our choice of the local standard deviation in (2.1) results in a roughly constant ́contribution of the Qs to the Reynolds stress and agrees with a similar choice in del Alamo et al. (2006) for the identification of vortex clusters.
As argued by Bogard & Tiederman (1986), quadrant analysis depends on the value of the hyperbolic-hole size H. They report that the optimum threshold for the buffer region is H ≈ 1, based on direct comparisons between the detected events and instantaneous visualizations of the flow. We choose H based on the percolation behaviour of (2.1). Percolation theory describes the statistics of the connected components of a random graph. Here, it is applied to the variation with H of the volume of the connected objects extracted by (2.1), as first used to identify vorticity and dissipation structures in isotropic turbulence by Moisy & Jime ́nez (2004) and in channelsbydelA ́lamoetal.(2006).

Figure 1(a) shows the percolation diagram of (2.1) in the two channels considered here. The solid lines are the ratio of the volume of the largest identified object, Vlar, to the total volume Vtot satisfying (2.1), and the dashed ones are the total number of identified objects, N/Nmax , normalized with its maximum over H. When H & 3, the identification only yields a few small objects that correspond to the strongest Qs. Decreasing H introduces new Qs, while previously identified ones grow in size. At first, the size of the largest Q changes little, and Vlar/Vtot stays roughly constant, but eventually the objects start to merge, resulting in a rapid increase of Vlar /Vtot and in a decrease of the number of objects. Figure 1(a) shows that this percolation crisis takes place in the range 0.5 . H . 3, independently of the Reynolds number. For lower thresholds, Vlar /Vtot ≈ 1, and most of the volume satisfying (2.1) belongs to a single object.

***
# The minimal flow unit in near-wall turbulence
***
> Jiménez, J., & Moin, P. (1991). The minimal flow unit in near-wall turbulence. Journal of Fluid Mechanics, 225, 213–240. doi:10.1017/S0022112091002033
> 
> Latex :
> > @article{Jiménez_Moin_1991, title={The minimal flow unit in near-wall turbulence}, volume={225}, DOI={10.1017/S0022112091002033}, journal={Journal of Fluid Mechanics}, author={Jiménez, Javier and Moin, Parviz}, year={1991}, pages={213–240}} <div></div>

#### Why study this paper ?

This paper is crucial because it serves as the origin of the minimal channel flow case, establishing it as the canonical state. Therefore, it is essential to thoroughly examine the setup provided in the paper.


## Variables
- $U$: Reference velocity.
- $h$: Cross-channel extent.
- $w^+$: Laminar value of vorticity at wall.
- $Q^+$: Time-averaged wall shears.
- $u^+$: Local friction velocities.
- $S^+$: Lengthscales.
- $t^+$: Local 'wall' time.
- $Re$: Reynolds number.
- $Ax^+$: Streamwise grid resolution.
- $Az^+$: Spanwise grid resolution.
- $T$: Various intrinsic timescales in the flow.

## Formulas
- Reynolds number: $Re = \frac{U h}{\nu}$ (Characterize flow).
- Time-averaged wall shears: $Q^+ = \overline{p' u} / (\rho U^2)$ (Primary diagnostic quantities).
- Local friction velocities: $u^+ = \frac{u}{U}$ (Characterize flow).
- Lengthscales in wall units: $S^+ = \frac{v}{Q^+}$ (Characterize flow).
- Wall time: $t^+ = \frac{t u^+}{S^+} = \frac{t}{\tau}$ (Minimum timescale for events in wall layer).
- Eddy turnover time: $T = O(1)$ (Intrinsic timescale).
- Sublayer viscous timescale: $O(l/Q)$ (Intrinsic timescale).
- Outer flow viscous timescale: $O(Re)$ (Intrinsic timescale).
- Intermediate timescale: $T = O(1000)$ (Intrinsic timescale).

## Computational Setup
<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/minimal_channel_domain.png" alt="Figure 1 du papier" width="700">
</div>

### Code Used
- Described by Kim, Moin & Moser (1987)
- Fully spectral Navier-Stokes integrator (DNS)
- De-aliased Fourier expansions in two homogeneous directions $(x, z)$
- Tchebichev polynomials in cross-channel $(y)$ coordinate

### Boundary Conditions
- Physical (no-slip) walls at top and bottom boundaries of computational box
- Periodic boundaries in streamwise and spanwise directions (represent the period, fundamental wavelength)
- Doubly infinite periodic array of structures with streamwise and spanwise periods $\lambda_x$ and $\lambda_z$ respectively, and cross-channel extent $2h$

#### Details 
In the context of this simulation setup, a "doubly infinite periodic array of structures" refers to the arrangement of flow structures within the computational domain. Let's break down the components:

- **Doubly Infinite**: This implies that the array of flow structures extends infinitely both in the streamwise $(x)$ and spanwise $(z)$ directions. In other words, if you were to move in either the $x$-direction or the $z$-direction, you would encounter identical flow structures repeating at regular intervals indefinitely.

- **Periodic Array**: Each flow structure within this array repeats itself periodically. This means that if you were to examine the flow at any given location within the domain, you would observe the same pattern of flow structures repeating at intervals determined by the streamwise and spanwise periods.

- **Streamwise and Spanwise Periods ($\lambda_x$ and $\lambda_z$)**: These represent the distances between repetitions of the flow structures in the streamwise $(x)$ and spanwise $(z)$ directions, respectively. For example, if $\lambda_x$ is the streamwise period, it means that the flow structures repeat every $\lambda_x$ units along the $x$-axis.

- **Cross-channel Extent $(2h)$**: This refers to the distance between the top and bottom boundaries of the computational domain in the cross-channel $(y)$ direction. The factor of 2 is included because it represents the full height of the channel, from the top wall to the bottom wall.

### Simulation Impositions
- Constant instantaneous volume flux in $x$-direction
  $$ Q = \frac{1}{\lambda_z} \int_{0}^{\lambda_z} \int_{-h}^{h} u \, dy \, dz = \frac{4}{3} hU $$
- Zero instantaneous averaged pressure gradient in **spanwise** direction (pressure is strictly periodic in $z$)
$$ P(z=0) = P(z=\lambda_z) $$
- Normalization: $U = 1$, $h = 1$ so laminar vorticity at wall: $w_z = 2$

### Diagnostic Quantities
- Streamwise pressure gradient and spatially averaged instantaneous wall shears used as primary diagnostics, the wall shears is define as follow :
$$ \Omega(z=\pm 1) = \frac{1}{\lambda_x \lambda_z} \int_{0}^{\lambda_x} \int_{0}^{\lambda_z} \frac{\partial u}{\partial y} \bigg|_{\left(x, \pm 1, z\right)} \, dx \, dz.
 $$
- Time-averaged wall shears: $\overline{\Omega}_{\tau}(z=\pm 1)$
- Local friction velocities: $u_{\tau}(z=\pm 1)=(\overline{\Omega}_{\tau}\nu)^{1/2}$
- Local lengthscales: $\delta_{\tau}(z=\pm 1)=(\nu/\overline{\Omega}_{\tau})^{1/2}$
- Refer velocity: $u^+=u/u_{\tau}$
- Refer length: $y^+=y/\delta_{\tau}$
- Local 'wall' time (minimum timescale for events in the wall layer): $t^+ = tu_{\tau}/\delta_{\tau} = \overline{\Omega}_{\tau}t$

  
### Reynolds Number
- Most convenient to characterize the flow
- $Re = \frac{Uh}{\nu} = \frac{3Q}{4\nu}$

### Computational Grid
- Grid resolution: $32\times 129\times16$ in $x,y,z$ ($48\times129\times24$ before de-aliasing)
- Spanwise period varied between 80 and 160 wall units, streamwise period between 300 and 600
- Corresponding grid resolution: $\Delta_z^+ = 5-10$, $\Delta_x^+ = 8-16$

###### TO CONTINUE 

### Numerical Validation
- Sample streamwise and spanwise power spectra provided
- Two-point spatial autocorrelation function for velocity fields demonstrated

### Time Integration
- Second-order Adams-Bashforth scheme for convective terms
- Implicit second-order Crank-Nicholson scheme for linear viscous terms
- Maximum CFL number kept at approximately 0.2
- Relative time-marching error: 0.1 per eddy turnover time

### Initial Conditions
- Started flow for each combination of $Re$ and wavelengths using fully developed flow field from nearby parameters
- First run initiated as random finite-amplitude perturbation of parabolic flow at $Re = 7000$
- Subsequent runs started incrementally from results of initial simulation

### Long Integration Times
- Intrinsic timescales in flow: eddy turnover time, sublayer viscous timescale, outer flow viscous timescale
- Additional intermediate timescale discovered during investigation
- Required running times of 1000-3000 to establish asymptotically steady flow
- Each time step required 0.7s on Cray Y-MP, requiring 50-100 CPU hours for sustained turbulence simulations

### Conclusion
The setup provides a comprehensive framework for conducting direct numerical simulations of spatially periodic channel flow, ensuring accurate representation of turbulent scales and robustness in capturing flow behavior across various Reynolds numbers and spatial resolutions.